In [1]:
%%shell

cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.pjoo07g7cE/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.YutPJVOAWK/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.6yESRqprxY/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [2]:
!pip install selenium
!apt-get update
!apt-get install chromium chromium-driver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:3 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:8 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Hit:9 http://archive.ubuntu.com/ubuntu

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [27]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [28]:
chrome_driver = web_driver()

In [29]:
YOUTUBE_TRENDING_URL = 'https://www.youtube.com/feed/trending'

def get_videos(driver):
  VIDEO_DIV_TAG = 'ytd-video-renderer'
  driver.get(YOUTUBE_TRENDING_URL)
  videos = driver.find_elements(By.TAG_NAME, VIDEO_DIV_TAG)
  # print(f"Found {len(videos)} videos")

  return videos

### Getting the video divs

In [30]:
videos = get_videos(chrome_driver)
print(f"Found {len(videos)} videos")

Found 99 videos


In [70]:
def parse_video(video):
  title_tag = video.find_element(By.ID, 'video-title')
  title = title_tag.text

  url = title_tag.get_attribute('href')

  thumbnail_tag = video.find_element(By.TAG_NAME, 'img')
  thumbnail_url = thumbnail_tag.get_attribute('src')

  channel_div = video.find_element(By.CLASS_NAME, 'ytd-channel-name')
  channel_name = channel_div.text

  upload_div = video.find_element(By.CLASS_NAME, 'ytd-video-meta-block')
  views_upload_span_tags = views_tag.find_elements(By.CLASS_NAME, 'inline-metadata-item')
  views = views_upload_span_tags[0].text

  upload_date = views_upload_span_tags[1].text

  description = video.find_element(By.ID, 'description-text').text

  return {
      'title': title,
      'url': url,
      'thumbnail_url': thumbnail_url,
      'channel': channel_name,
      'description': description
  }

In [72]:
# Parsing top ten videos
videos_data = [parse_video(video) for video in videos[:10]]
print(videos_data)

[{'title': 'Iniko - Jericho (Official Visualizer)', 'url': 'https://www.youtube.com/watch?v=Vc75pr0UteQ', 'thumbnail_url': 'https://i.ytimg.com/vi/Vc75pr0UteQ/hqdefault.jpg?sqp=-oaymwE2CPYBEIoBSFfyq4qpAygIARUAAIhCGAFwAcABBvABAfgB_gmAAtAFigIMCAAQARg3IGAoZTAP&rs=AOn4CLCY20h7lduIbsgs6QxJHSv9h1hIbg', 'channel': 'Iniko', 'description': 'Official visualizer for “Jericho” by Iniko. Listen to “Jericho” here: https://iniko.lnk.to/Jericho Follow Iniko: YouTube - https://www.youtube.com/@Iniko TikTok - https://www.tiktok.com/@...'}, {'title': 'WE BOUGHT SMOSH!', 'url': 'https://www.youtube.com/watch?v=DwsZggwPRKc', 'thumbnail_url': 'https://i.ytimg.com/vi/DwsZggwPRKc/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFfyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLAFguFzUJX_CcVFm0SoxKpVdY2lpg', 'channel': 'Smosh', 'description': "There's Anthony! BECOME A MEMBER TODAY to support a free and independent Smosh. https://youtube.com/smosh/join SUBSCRIBE: https://smo.sh/Sub2Smosh LIMITED EDITION SMOSH REUNION T-SHIRT!:..."},

Save to csv

In [76]:
import pandas as pd

videos_df = pd.DataFrame(videos_data)
# videos_df.head(10)
videos_df.to_csv('trending.csv', index=None)

In [77]:
videos_df.head(10)

,title,url,thumbnail_url,channel,description
0,Iniko - Jericho (Official Visualizer),https://www.youtube.com/watch?v=Vc75pr0UteQ,https://i.ytimg.com/vi/Vc75pr0UteQ/hqdefault.j...,Iniko,Official visualizer for “Jericho” by Iniko. Li...
1,WE BOUGHT SMOSH!,https://www.youtube.com/watch?v=DwsZggwPRKc,https://i.ytimg.com/vi/DwsZggwPRKc/hqdefault.j...,Smosh,There's Anthony! BECOME A MEMBER TODAY to supp...
2,Goodbye…,https://www.youtube.com/watch?v=9ZqaXmjpTr8,https://i.ytimg.com/vi/9ZqaXmjpTr8/hqdefault.j...,Tfue,I can’t put into words how grateful and thankf...
3,CHALLENGERS | Official Trailer,https://www.youtube.com/watch?v=MDnVk5jIJr0,None,MGM,The art of seduction and other games. Watch th...
4,DRAGON BALL LEGENDS REVEALS ＆ STUFF: SPECIAL E...,https://www.youtube.com/watch?v=611TpZ9miv8,None,Bandai Namco Entertainment,There was a mistake with one of the numbers th...
5,AMP MMA BASKETBALL,https://www.youtube.com/watch?v=ZSnGqzaTNlc,None,AMP,SHOUTOUT @PFLMMA FOR THE FACILITY HOOK UP! SUB...
6,Coast Guard gives update on missing submarine ...,https://www.youtube.com/watch?v=bCDAEF-JVkY,None,CBS Boston,The Coast Guard shared an update Monday on the...
7,This Storm Will Literally Hit Different…,https://www.youtube.com/watch?v=Oca-qcU5zoQ,None,"Ryan Hall, Y'all",You’ll be amazed at what you can do with Gramm...
8,Rocky Aur Rani Kii Prem Kahaani - OFFICIAL TEA...,https://www.youtube.com/watch?v=HZAyjBMTiaM,None,Dharma Productions,"Karan Johar, in his 25th anniversary year - in..."
9,"Bradley Beal traded to Suns, Draymond opts-out...",https://www.youtube.com/watch?v=F3oL7jJQKjw,None,The Herd with Colin Cowherd,Ric Bucher joins Colin Cowherd to discuss the ...
